In [1]:
import csv
import numpy as np
import pandas

# open the file in universal line ending mode 
with open('post.csv', 'r') as post_file:
  # read the file as a dictionary for each row ({header : value})
  reader = csv.DictReader(post_file)
  post = {}
  for row in reader:
    for header, value in row.items():
      try:
        post[header].append(value)
      except KeyError:
        post[header] = [value]
        
#Put all the parameters in separate lists 
uid = post['id']
pid = post['pid'] 
shares = post['shares']
msg = post['msg'] 

l_pid = len(pid)  #the number of the  post id's in the network


# open the file in universal line ending mode 
with open('post_RT.csv', 'r') as post_RT_file:
  # read the file as a dictionary for each row ({header : value})
  reader = csv.DictReader(post_RT_file)
  prt = {}
  for row in reader:
    for header, value in row.items():
      try:
        prt[header].append(value)
      except KeyError:
        prt[header] = [value]

#Put all the parameters in separate lists     
post_id = prt['post_id']
RT = prt['Response Trust']

l_post = len(post_id)  #the number of the post id's in the network

RTU = np.asarray(RT, dtype = float)  #Convert the contents of the 'RT' list, into the numeric array 'RTU' for calculations

S = []
U = []

for i in range(l_pid):
    s=0
    for k in range(l_post):
        if pid[i] == post_id[k]:  #Search for the posts(post id's) that exist in both files
            s = s + RTU[k]  #Calculates the RESPONSE TRUST of all the posts, that belong to each user
            
    S.append(s)  #Append the RESPONSE TRUST of each user we found, into the list 'S'
    U.append(uid[i])  #Append the user id, to which the RESPONSE TRUST we found, into the list 'U'
        
#for i in range(l_pid):
    #print(U[i],pid[i],S[i],sep=',')
        
User = []
S1 = []
S2 = []
s3 = []
S3 = []
count = 0

S1 = np.asarray(S, dtype = float)  #Convert the contents of the 'S' list into the numeric array 'S1' for calculations

for i in range(l_pid):
    s2 = S1[i]
    for k in range(i+1, l_pid):
        if U[i] == U[k] and i!=k :  #Search for the different user id's in the file
            U[k] = ''
            s2 = s2 + S1[k]  #Calculate the total RESPONSE TRUST of the user
     
    if U[i] != '':
        count = count + 1  #Counts all the different users in the network
        User.append(U[i])  #Append the unique user id into the list 'User'
        s3 = round(s2,2)   #Store the,rounded by 2 decimal places, total RESPONSE TRUST of each unique user, into the 's3' list
        S3.append(s3)   #Append the contents into the 'S3' list



# open the file in universal line ending mode 
with open('user_PT.csv', 'r') as user_PT_file:
  # read the file as a dictionary for each row ({header : value})
  reader = csv.DictReader(user_PT_file)
  u_post = {}
  for row in reader:
    for header, value in row.items():
      try:
        u_post[header].append(value)
      except KeyError:
        u_post[header] = [value]
        
#Put all the parameters in separate lists 
user_posts = u_post['user_posts']
user_PT = u_post['Participation Trust']

lp = len(user_posts)  #the number of the posts of each user in the network


                   # AVERAGE RESPONSE TRUST 

USER_RT = np.asarray(S3, dtype = float)  #Convert the contents of the 'S3' list above, into a numeric array for calculations
USER_POSTS = np.asarray(user_posts, dtype = int)  #Convert the contents of the 'use_posts' list of the file, into a numeric array for calculations
avg = []
avg1 = []
AVG = []

for i in range(lp):
    avg = USER_RT[i] / USER_POSTS[i]  #Calculate the AVERAGE RESPONSE TRUST for each user 
    avg1 = round(avg,4)  #Store the, rounded by 2 decimal places, AVERAGE RESPONSE TRUST for each user 
    AVG.append(avg1)   #Append the result of the AVERAGE RESPONSE TRUST we found for each user, into the 'AVG' list 
    
    
                 # INDIRECT TRUST

USER_PT = np.asarray(user_PT, dtype = int)  #Convert the contents of the 'user_PT' list of the file, into a numeric array for calculations
USER_AVG = np.asarray(AVG, dtype = float)   #Convert the contents of the 'AVG' list above, into a numeric array for calculations
t = []
t1 = []
T = []

for i in range(lp):
    t = 0.5 * USER_PT[i] + 0.5 * USER_AVG[i]   #Calculate the INDIRECT TRUST for each user
    t1 = round(t,4)   #Store the, rounded by 2 decimal places, INDIRECT TRUST for each user
    T.append(t1)   #Append the final result of the INDIRECT TRUST for each user, into the list 'T'
        

with open('INDIRECT_TRUST.csv', mode='w') as it_file:
    fieldnames = ['user_id','user_posts' ,'Participation Trust','Response Trust','Indirect Trust']
    writer = csv.DictWriter(it_file, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(lp):
        writer.writerow({'user_id': User[i], 'user_posts': USER_POSTS[i], 'Participation Trust': user_PT[i] , 'Response Trust': AVG[i], 'Indirect Trust': T[i]})      
    
    
#df = pd.read_csv("new_post.csv")
#df["weights"] = S1
#df.to_csv("new_post.csv", index=False)